<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/Neural_networks/NLP/Langchain/Langchain_llms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLMs with Langchain and RAGs

## Installs and imports

In [1]:
!pip install -q torch langchain bitsandbytes accelerate transformers sentence-transformers faiss-gpu pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 18.4 MB/s eta 0:00:00


In [2]:
import os
import torch
import transformers

from glob import glob
from tqdm.notebook import tqdm
from transformers import AutoTokenizer,AutoModelForCausalLM,BitsAndBytesConfig,pipeline
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.schema.output_parser import StrOutputParser

import warnings
warnings.simplefilter('ignore')

## Config

In [ ]:
device = 'cuda'
modelname = "mistralai/Mistral-7B-v0.1"
embedder = 'sentence-transformers/all-mpnet-base-v2'

## Download data

In [ ]:
!wget https://media.wizards.com/2018/dnd/downloads/DnD_BasicRules_2018.pdf
!wget https://winghornpressdotcom.files.wordpress.com/2018/02/the_wild_sheep_chase_v2.pdf
!wget https://turtlegodsclan.weebly.com/uploads/3/0/5/8/30584041/a_dark_and_stormy_knight.pdf

## Process data into chunks

In [ ]:
pdfs = glob("*.pdf")
docs = []

progress_bar = tqdm(total=len(pdfs), desc="Processing")

for path in pdfs:
  loader = PyPDFLoader(path)
  doc = loader.load()

  splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=8)
  chunks = splitter.split_documents(doc)
  docs.extend(chunks)
  progress_bar.update(1)

progress_bar.close()

## Set quantization to 4-bit low-precision parameters

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False)

## Load model and tokenizer

In [ ]:
model = AutoModelForCausalLM.from_pretrained(modelname, quantization_config=bnb_config,
                                             do_sample=True, device_map = "auto")

tokenizer = tokenizer = AutoTokenizer.from_pretrained(modelname)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

## Set promting pattern

In [ ]:
text_generation_pipeline = pipeline(
                                    model=model,
                                    tokenizer=tokenizer,
                                    task="text-generation",
                                    temperature=0.5,
                                    repetition_penalty=1.2,
                                    return_full_text=True,
                                    max_new_tokens=400)

mistral = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [9]:
prompt_template = """
Instruction: You are an experienced Dungeon Master. You provide information about the game and advice players.
Answer the question based on the following context:
{context}

Question:
{question}
 """

prompt = PromptTemplate(input_variables=["context", "question"], template=prompt_template)

## Init chain

### init prompt->llm

In [ ]:
llm_chain = LLMChain(llm=mistral, prompt=prompt)

### Init db

In [ ]:
db = FAISS.from_documents(docs, HuggingFaceEmbeddings(model_name=embedder))

### init db retriever

In [ ]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={'k': 3})

### Init RAG chain

In [ ]:
rag_chain = ({"context": retriever, "question": RunnablePassthrough()} | llm_chain)

## Test result

In [16]:
query = "What is the best class and race if i want to play a sneaky but magical character?"
response = rag_chain.invoke(query)

print ("Question:", response["question"])
print (response["text"].replace('\\n', '\n'))